In [31]:
# -*- coding: utf-8 -*-
import fitz  # PyMUPDF
import os
import unidecode
import re
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode
from pylatexenc.latex2text import LatexNodes2Text


class LocalSearch:
    directory = None
    specialCharacters = ['´', '¸', '˜', '`', '^']
    questionFolders = None
    def __init__(self, path):
        assert path != None, 'None path'
        assert isinstance(path,str), 'Path must be a string'
        assert path , 'Path must be not empty'
        
        self.questionFolders = [x for x in os.listdir(self.directory)]
    
    def __decodePdf(self,path):
        doc = fitz.open(path)
        for page in doc:
            text = page.getText()
        return text


    def __decodeTex(self,path):
        doc = " ".join(open(path, "r").readlines()).encode("ISO-8859-1").decode("utf-8")
        return LatexNodes2Text().latex_to_text(doc)

    def __decodePy(self,path):
        doc = open(path,'r')
        return doc.read()

    def __removeSpecialChar(self,text):
        for sChar in self.specialCharacters:
            text = text.replace(sChar, '')

        return text


    def searchKeyword(self,keyword):
        pdfResult = []

        for question in self.questionFolders:
            text = ""
            path = self.directory + "\\" + question + "\\version_1"
    
            if docType == 'pdf':
                path += "\\true_or_false_question.pdf"
                text = self.__decodePdf(path)
            
            elif docType == 'latex':
                path += "\\true_or_false_question.tex"
                text = self.__decodeTex(path)

            elif docType == 'python':
                path += "\\program.py"
                text = self.__decodePy(path)

            else:
                return ["Error - wrong Document Type"]

            keyword = unidecode.unidecode(keyword)

            if re.search(keyword, text, flags=re.IGNORECASE) is not None:
                pdfResult.append(path)

        return pdfResult

# ls = LocalSearch("C:\\Users\\ASUS\\Desktop\\Semestre6\\Projeto\\perguntas")
ls = LocalSearch('')
print(ls.searchKeyword("pseudo_random_integer",'python'))


AssertionError: Path must be not empty